In [1]:
%pip install -q -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
print(load_dotenv('../../.env'))
print(os.environ['LANGSMITH_PROJECT'])
os.environ['LANGSMITH_TRACING']="true"
os.environ['USER_AGENT'] = 'myagent'
QDRANT_URL="http://localhost:6333"

True
agentic-ops


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

retriever = QdrantVectorStore(
            client=QdrantClient(url=QDRANT_URL),
            collection_name="documentations",
            embedding=OpenAIEmbeddings(model="text-embedding-3-small")
        ).as_retriever()

# Threads

In [4]:
import uuid
thread_id = uuid.uuid4()

In [6]:
from langsmith import traceable
from openai import OpenAI
from typing import List

openai_client = OpenAI()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


https://smith.langchain.com/

## Running twice to show 2 turns

In [7]:
question = "What is huggingface ?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

Hugging Face is a company that provides a platform for sharing and collaborating on machine learning models, particularly in natural language processing. They offer the Model Hub, where users can upload, download, and version models, as well as tools like the `huggingface_hub` library for programmatic interaction with the Hub. Hugging Face promotes open-source collaboration to democratize artificial intelligence.


In [8]:
question = "what are datasets in huggingface then ? "
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

Hugging Face datasets are collections of data that are stored on disk and can be easily accessed and manipulated for machine learning tasks. They allow users to load, preprocess, and stream data efficiently, reducing memory usage by not loading the entire dataset into memory at once. These datasets can be used with various models and frameworks, including TensorFlow and PyTorch.


## Traces UI tab